# PAGE - CONTRIBUTION

**Définition** : 

Ce notebook permet de générer les KPIs autours des pages contributions :

 * Convertion des contributions génériques : Taux de personnes ayant sélectionné une CC depuis une contribution générique


## Récupération des données

In [1]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

### Interval

Renseignez la date de début (incluse) et de fin (non incluse) souhaitées pour la récupération des données

In [2]:
date_debut = '2024-10-01' # incluse
date_fin = '2024-10-31' # exclu

### Requête ES

In [3]:
es_connector = ElasticsearchConnector(env='monolog')
QUERY = {
  "query": {
    "bool": { 
      "must": [
        {
          "regexp": {
            "url": "https://code\\.travail\\.gouv\\.fr/contribution/[a-zA-Z]+.*" 
          }
        },
        {
          "range": {
            "logfile": {
              "gte": date_debut,
              "lt": date_fin
            }
          }
        },
        {
          "terms": {
            "type": [
              "cc_select_p1",
              "cc_select_p2",
              "visit_content",
              "cc_select_traitée"
            ]
          }
        }
      ]
    }
  }
}

Connection with Elasticsearch established with success


In [4]:
logs = es_connector.execute_query(QUERY, "logs-new")

  0%|          | 0/268939 [00:00<?, ?it/s]

## Nombres de visites par contribution générique

### Préparation des données

On nettoie les urls pour éviter des duplicats sur les contributions.

In [5]:
# On duplique les données pour éviter d'altérer la source
logs_contrib = logs.copy()

# Nettoyer les URLs pour retirer tout ce qui suit "?"
logs_contrib['cleaned_url'] = logs_contrib['url'].str.split('?').str[0]  # Retire tout ce qui suit "?"
logs_contrib['cleaned_url'] = logs_contrib['cleaned_url'].str.split('#').str[0]  # Retire tout ce qui suit "#"

### Ratio de complétion par contribution

On va grouper par URL afin d'avoir le nombre de visites, de sélection de la convention collective par l'entreprise ou directement. On va ainsi pouvoir calculer le ratio pour chaque contribution.
Les données sont ensuite exportés dans le fichier ratio_contributions.csv

In [11]:
# Grouper par la colonne nettoyée 'cleaned_url'
grouped_contribs = logs_contrib.groupby('cleaned_url').apply(
  lambda group: pd.Series({
        'cc_select_p1': (group[group['type'] == 'cc_select_p1'].drop_duplicates(subset='idVisit')).shape[0],
        'cc_select_p2': (group[group['type'] == 'cc_select_p2'].drop_duplicates(subset='idVisit')).shape[0],
        'cc_select_traitée': (group[group['type'] == 'cc_select_traitée'].drop_duplicates(subset='idVisit')).shape[0],
        'visits': (group[group['type'] == 'visit_content'].drop_duplicates(subset='idVisit')).shape[0]
    })
).reset_index()

# Ajouter la colonne pour le ratio
stats = grouped_contribs
stats['cc_select_ratio'] = ((stats['cc_select_p1'] + stats['cc_select_p2']) / stats['visits'] * 100)
stats['cc_traitée_ratio'] = (stats['cc_select_traitée'] / (stats['cc_select_p1'] + stats['cc_select_p2']) * 100)
# Remplacer les valeurs infinies ou NaN par 0 (exemple si visit_content_count est 0)
stats['cc_select_ratio'] = stats['cc_select_ratio'].fillna(0)
stats['cc_traitée_ratio'] = stats['cc_traitée_ratio'].fillna(0)

stats = stats.rename(columns= {
    'cleaned_url': 'URL',
    'cc_select_p1': 'Sélection via CC (P1)',
    'cc_select_p2': 'Sélection via entreprise (P2)',
    'cc_select_traitée': 'CC traitée',
    'visits': 'Visites uniques',
    'cc_select_ratio': 'Ratio de conversion',
    'cc_traitée_ratio': 'Ratio de CC traitée'
})


/var/folders/91/m2qzzht5131f_67601_c_0l40000gn/T/ipykernel_56856/3327623547.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_contribs = logs_contrib.groupby('cleaned_url').apply(


In [13]:
# Trie par ratio
stats = stats.sort_values(by='Visites uniques', ascending=False)

styled_df = (
    stats.style
    .format({"Ratio de conversion": "{:.2f}%"})
    .format({"Ratio de CC traitée": "{:.2f}%"})
)

styled_df

,URL,Sélection via CC (P1),Sélection via entreprise (P2),CC traitée,Visites uniques,Ratio de conversion,Ratio de CC traitée
12,https://code.travail.gouv.fr/contribution/en-cas-darret-maladie-du-salarie-lemployeur-doit-il-assurer-le-maintien-de-salaire,4972,1967,4732,21550,32.199536,68.19%
28,https://code.travail.gouv.fr/contribution/les-conges-pour-evenements-familiaux,5337,1680,4551,21416,32.765222,64.86%
21,https://code.travail.gouv.fr/contribution/la-periode-dessai-peut-elle-etre-renouvelee,3262,760,2908,9969,40.345070,72.30%
3,https://code.travail.gouv.fr/contribution/a-quelles-indemnites-peut-pretendre-un-salarie-qui-part-a-la-retraite,3097,767,2537,9007,42.899967,65.66%
36,https://code.travail.gouv.fr/contribution/quelle-est-la-duree-du-preavis-en-cas-de-demission,1452,826,1542,7373,30.896514,67.69%
52,https://code.travail.gouv.fr/contribution/si-le-salarie-est-malade-pendant-ses-conges-quelles-en-sont-les-consequences,1803,479,1533,6779,33.662782,67.18%
31,https://code.travail.gouv.fr/contribution/quand-le-salarie-a-t-il-droit-a-une-prime-danciennete-quel-est-son-montant,1404,603,1343,5557,36.116610,66.92%
48,https://code.travail.gouv.fr/contribution/quelles-sont-les-consequences-du-non-respect-du-preavis-par-le-salarie-ou-lemployeur,1059,393,1111,5170,28.085106,76.52%
23,https://code.travail.gouv.fr/contribution/le-preavis-de-demission-doit-il-etre-execute-en-totalite-y-compris-si-le-salarie-a-retrouve-un-emploi,821,336,951,4847,23.870435,82.20%
7,https://code.travail.gouv.fr/contribution/comment-determiner-lanciennete-du-salarie,1385,224,1118,4646,34.631941,69.48%


### Sauvegarde du tableau dans un fichier CSV

Le fichier est disponible dans le dossier `explorations/outputs`

In [8]:
stats.to_csv(f"./outputs/ratio_completion_contribution_{date_debut}_{date_fin}.csv", index=False, encoding='utf-8')